In [1]:
### Load
%matplotlib inline
import os
import matplotlib.pyplot as plt
import numpy as np
import random 
import pandas as pd
from ggplot import *
%load_ext rmagic

pupil_params = np.array([500,500,1500,2000]) 
import pymc3 as pm
import theano
import theano.tensor as T
from pymc3.distributions.timeseries import GaussianRandomWalk
import scipy
import spectral


/Library/Python/2.7/site-packages/IPython/extensions/rmagic.py:693: UserWarning: The rmagic extension in IPython is deprecated in favour of rpy2.ipython. If available, that will be loaded instead.
http://rpy.sourceforge.net/
  warnings.warn("The rmagic extension in IPython is deprecated in favour of "
:0: FutureWarning: IPython widgets are experimental and may change in the future.


In [ ]:
%%R
library(glmnet)
cvfit = cv.glmnet(X.train, Y.train[,1])
plot(cvfit)
#lm.ridge(Y.train[,1] ~ X.train,lambda=0.1)
#select(lm.ridge(Y.train[,1] ~ X.train, lambda = seq(0,20,0.001)))
cvfit$lambda.min
#predict(cvfit, newx = X.test, s = "lambda.min")

## Showing Cancor in R is gives the Q matrix from BF

In [2]:
%%R
n=40
X = round(mvrnorm(n,mu=matrix(0,4),diag(4),4))
Y = round(mvrnorm(n,mu=matrix(0,3),diag(3),4))
          
cc=cancor(X,Y)
Q=solve(t(Y)%*%Y)%*%(t(Y)%*%X)%*%solve(t(X)%*%X)%*%(t(X)%*%Y)
print(Q)
print(cc)

# how to get Q back from the R function
C=(diag(3)*(cc$cor**2))
#print(C)
print(t(cc$ycoef)%*%C%*%t(solve(cc$ycoef)))
print(cc$ycoef%*%C%*%solve(cc$ycoef))

ERROR: Cell magic `%%R` not found.


## Why is Matlab's output different??

In [1]:
%%R -o X,Y
Z = matrix(.9,2,2)
diag(Z)=1
out<-mvrnorm(100,matrix(0,2),Z)
y1<-out[,1]
x<-out[,2]
#plot(x,y1)
y2 <-rnorm(100,0,1)
print(cor(x,y1))
#plot(x,y1)
x2 <-rnorm(100,0,1)

Y = cbind(y1,y2)
X = cbind(x,x2)
cc<-cancor(X,Y)
print(cc)
scatterplot3d(y1,y2,x)

Yn = t(cc$ycoef%*%t(Y))
y1n=Yn[,1]
y2n=Yn[,2]
#print(Yn)
scatterplot3d(y1n,y2n,x)

ERROR: Cell magic `%%R` not found.


In [ ]:
import sklearn.cross_decomposition as cd
cca=cd.CCA()
c=cca.fit(np.array(X),np.array(Y))
#c=c.get_params()
#print(c)
print(c.y_loadings_)
print(c.y_weights_)
print(c.y_rotations_)
X_train_r, Y_train_r = cca.transform(np.array(X),np.array(Y))

cca.score(np.array(X),np.array(Y))
#c=c.get_params()x<-c(1,1,4,2,1,4)
y1<-c(4,1,4,2,6,4)
y1<-c(4,3,5,6,1,1)
Y = cbind(y1,y2)
X = cbind(x,x2)
# interesting you can keep fewer dimension.. scikit uses it as dimension reduction.. 

In [23]:
%%R

x<-c(1,1,4,2,1,4)
y1<-c(4,1,4,2,6,4)
y2<-c(4,3,5,6,1,1)
Y<-cbind(y1,y2)
X = x
c<-cancor(X,Y,xcenter=FALSE,ycenter=FALSE)
c

$cor
[1] 0.8447087

$xcoef
          [,1]
[1,] 0.1601282

$ycoef
         [,1]       [,2]
y1 0.06695451  0.1300887
y2 0.04837141 -0.1389584

$xcenter
[1] 0

$ycenter
[1] 0 0



In [26]:
%%R
Q=solve(t(Y)%*%Y) %*% (t(Y)%*%X) %*% solve(t(X)%*%X) %*% (t(X)%*%Y)
Q
### YX XY / YY XX basically 
### 
out<-eigen(Q)
print(out)
r<-sqrt(out$values)
print(svd(Q))

$values
[1]  7.135328e-01 -5.551115e-17

$vectors
          [,1]       [,2]
[1,] 0.8105909 -0.6834243
[2,] 0.5856128  0.7300214

$d
[1] 7.193083e-01 4.052433e-17

$u
           [,1]       [,2]
[1,] -0.8105909 -0.5856128
[2,] -0.5856128  0.8105909

$v
           [,1]       [,2]
[1,] -0.7300214  0.6834243
[2,] -0.6834243 -0.7300214



Why does R have only 4coefficients and matlab 2
oh interesting R gives you 2x2 when x=2,y=2..
and my Q.. is 2x2 when I do it.. but perhaps matlab only returns the first 1 and R just does eigen.. 


## Signal Matrix

Calculating Signal

In [3]:
%%R
# each voxel shares a design matrix 
x1 = rep(c(-.25,-.25,-.25,-.25,1),8)
x2 = rep(c(-.66,-.66,-.66,.5,.5),8)
x1 = rnorm(40)
x2 = rnorm(40)
X = cbind(x1,x2)

b1= rnorm(3)
b2= rnorm(3)
B = cbind(b1,b2)
B = t(B)
#B=cbind(c(.4,.4),c(.1,.8),c(-.2,.3)) #  each voxel responds the same to the design matrix 
F=t(B)%*%t(X)%*%X%*%B
print(F)
solve(F)


ERROR: Cell magic `%%R` not found.


## Mscl fMRI Stuff

In [4]:
import nibabel as nib
import os
os.chdir('/Users/chris/Desktop/final_project')
import numpy as np
%load_ext rmagic
epi_img = nib.load('example_sub.nii.gz')
epi_img_data = epi_img.get_data()
epi_img_data.shape

NameError: name 'nib' is not defined

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
def show_slices(slices):
    """ Function to display row of image slices """
    fig, axes = plt.subplots(1, len(slices),figsize=(18,12))
    for i, slice in enumerate(slices):
        axes[i].imshow(slice.T, cmap="gray", origin="lower",interpolation='none')
        

        

slice_1 = epi_img_data[:, :, 10,1]
slice_2 = epi_img_data[:, :, 16,1]
epi_img_data[40,40,20,1]=2000
epi_img_data[40,42,20,1]=2000
epi_img_data[42,40,20,1]=2000
epi_img_data[44,40,20,1]=2000
epi_img_data[42,42,20,1]=2000
epi_img_data[44,42,20,1]=2000
slice_3 = epi_img_data[:, :, 22,1]
slice_4 = epi_img_data[:, :, 25,1]
show_slices([slice_1, slice_2,slice_3,slice_4])
plt.suptitle("Center slices for EPI image")  

print(np.max(epi_img_data[:, :, 20,1]))

In [ ]:
## make mask ##
sample_image =epi_img_data[:, :, 22,1]
mask=sample_image>350
mask[:,0:10]=0
plt.imshow(mask)
np.savetxt('underlay.txt',sample_image)
np.savetxt('mask.txt',mask)

#mask.to_filename('mask.nii')
#nib.save(mask,'mask.nii')

#sample_image.to_filename('underlay.nii')
#nib.save(sample_image,'underlay.nii')

## Mscl Fitting

generate data... 

In [ ]:
%%R

data.train<-generate_data()
data.test<-generate_data()

library(scatterplot3d)


par(mfrow=c(2,2))
scatterplot3d(data.train$Y)
title('Y space')

plot(data.train$Y[,1],type='l',col='red')
lines(data.train$Y[,2],type='l',col='blue')
lines(data.train$Y[,3],type='l',col='green')
title('Y across Time')

plot(data.train$X[,1],col='yellow',type='l')
lines(data.train$X[,2],col='orange',type='l',linestyle='--')
title('X across Time')


In [ ]:
%%R -w 880 -h 480 

# let's esimate

OLS_estimate<-function(data){
    Bhat.ols = solve(t(X)%*%X)%*%t(X)%*%Y
    Yhat = data$X%*%Bhat.ols
    MSE = mean((Yhat-data$Y)**2)
    fit = list(Yhat=Yhat,Bhat.ols=Bhat.ols,MSE=MSE)
    return(fit)
}
OLS_predict<-function(fit,data){
    Yhat = data$X%*%fit$Bhat.ols
    MSE = mean((Yhat-data$Y)**2)
    predict = list(MSE=MSE,Yhat=Yhat)
    return(predict)
}


train.fit.ols = OLS_estimate(data.train)
test.predicted = OLS_predict(train.fit.ols,data.test)

par(mfrow=c(1,2))
plot(data.train$Y[,1],type='l',col='red')
lines(train.fit.ols$Yhat[,1],type='l',col='red',lty=2)
lines(data.train$Y[,2],type='l',col='blue')
lines(train.fit.ols$Yhat[,2],type='l',col='blue',lty=2)
lines(data.train$Y[,3],type='l',col='green')
lines(train.fit.ols$Yhat[,3],type='l',col='green',lty=2)
title(paste('Training Set',train.fit.ols$MSE))

plot(data.test$Y[,1],type='l',col='red')
lines(test.predicted$Yhat[,1],type='l',col='red',lty=2)
lines(data.test$Y[,2],type='l',col='blue')
lines(test.predicted$Yhat[,2],type='l',col='blue',lty=2)
lines(data.test$Y[,3],type='l',col='green')
lines(test.predicted$Yhat[,3],type='l',col='green',lty=2)
title(paste('Test Set',test.predicted$MSE))


MSE.ols = matrix(0,1000)
for (sim in seq(1000)){
    data.train<-generate_data()
    data.test<-generate_data()
    train.fit.ols = OLS_estimate(data.train)
    test.predicted = OLS_predict(train.fit.ols,data.test)
    MSE.ols[sim] = test.predicted$MSE
    
}
hist(MSE.ols,main='Test MSE OLS')
mean(MSE.ols)

#### Prediction steeper slope

In [ ]:
%%R

data.train<-generate_data()
data.test<-generate_data()
train.fit.ols = OLS_estimate(data.train)
test.predicted = OLS_predict(train.fit.ols,data.test)
    
#plot(train.fit.ols$Yhat,data.test$Y)
abline(lm(data.test$Y~train.fit.ols$Yhat),col='green')
abline(a=0,b=train.fit.ols$Bhat.ols[1])

for (sim in seq(10)){
    data.train<-generate_data()
    data.test<-generate_data()
    train.fit.ols = OLS_estimate(data.train)
    test.predicted = OLS_predict(train.fit.ols,data.test)
    
    lines(train.fit.ols$Yhat,data.test$Y)
    abline(lm(data.test$Y~train.fit.ols$Yhat),col='green')
    abline(a=0,b=train.fit.ols$Bhat.ols[1])
}

In [ ]:
## convert out of R form
response_masked = np.array(response_masked)
response_masked[response_masked==0]=np.nan

avg_gaus_image = np.array(avg_gaus_image)
avg_gaus_image[avg_gaus_image==0]=np.nan


### Plot 
plt.figure(figsize=(18,12))
im1=plt.imshow(sample_slice.T, cmap="gray", origin="lower",interpolation='none')

import matplotlib.cm as cm
colmap = cm.get_cmap('hot')
np.shape(colmap._lut)
colmap._lut[:,-1]=np.ones(colmap.N+3)*.5
im2=plt.imshow(avg_gaus_image.T, cmap=colmap,origin="lower",interpolation='none')
plt.colorbar()
im3=plt.imshow(response_masked.T, cmap=colmap,origin="lower",interpolation='none')
plt.colorbar()
plt.show()